In [ ]:
from model import WhisperSegmenter, WhisperSegmenterFast
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm
from copy import deepcopy
from train import validate
from datautils import get_audio_and_label_paths
import os

In [ ]:
segmenter = WhisperSegmenter(  model_path = "nccratliri/whisperseg-zebra-finch", 
                        device = "cuda")

In [ ]:
fname_list = sorted( [fname for fname in os.listdir("data/dataset/zebseg/") if fname != "all_birds"] )

In [ ]:
for fname in fname_list:
    audio_paths, label_paths = get_audio_and_label_paths("data/dataset/zebseg/%s/test/"%(fname))

    audio_list, label_list = [], []
    for audio_path, label_path in zip(audio_paths, label_paths):
        audio, _ = librosa.load( audio_path, sr = 16000 )
        label_df = pd.read_csv( label_path )
        label = {
            "onset":list(label_df["onset"]),
            "offset":list(label_df["offset"]),
            "cluster":list(label_df["cluster"]) 
        }
        audio_list.append(audio)
        label_list.append(label)
    
    
    res = validate( audio_list, label_list, segmenter, sr = 16000, tolerance = 0.02, num_trials = 3 , min_segment_length = 0.02,
              voting_time_step = 1.0, voting_precision = 0.001, batch_size = 8, max_length = 400, eps = 0.02, target_cluster = None )
    print(fname+":\t", res)
    